In [2]:
import pandas as pd
import numpy as np
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
import enchant
from enchant.checker import SpellChecker

In [3]:
df = pd.read_csv('../data/raw_data.csv')

# Making text more uniform

Remove urls

In [5]:
df.body = df.body.replace(r'http\S+', '', regex=True)

Fix as many spelling errors as possible

In [6]:
spell_checker = SpellChecker("en_UK","en_US")

In [7]:
# the ’ was causing issues, took a while to notice
df.body = df.body.str.replace('’', '\'')

In [8]:
def correct_error(body):
    spell_checker.set_text(body)
    for err in spell_checker:
        if len(err.suggest())>0: 
            sug = err.suggest()[0]
            err.replace(sug)
    return spell_checker.get_text()

In [9]:
df.body = df.body.apply(lambda row: correct_error(row))

Remove stop words and replace negations with "not"

In [14]:
df.body = df.body.str.lower()

In [16]:
stopwords_list = stopwords.words('english')
negations = ['don\'t', 'aint' 'aren\'t', 'couldn\'t','didn\'t', 
             'doesn\'t', 'hadn\'t', 'hasn\'t', 'haven\'t', 'isn\'t', 
             'mightn\'t', 'mustn\'t', 'needn\'t', 'shouldn\'t', 'wasn\'t', 
             'weren\'t', 'won\'t', 'wouldn\'t', 'nor', 'not']
stopwords_list = [el for el in stopwords_list if el not in negations]
missing_words = ['i\'m', 'i\'d']
stopwords_list.extend(missing_words)

In [17]:
regx = r'\b(?:{})\b'.format('|'.join(negations))
df.body = df.body.str.replace(regx, 'not')

In [18]:
df.body = df.body.apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_list)]))

In [20]:
df.head(10)

,body,positive,negative,neutral,rated,comment_id,video_id,date
0,"*stretched past 10 minute mark ad revenue, see!*",0,1,0,1,UgztP4lVR-Epv5HlSXN4AaABAg,ItYOdWRo0JY,2020-01-10T20:24:33Z
1,big time scam,0,1,0,1,UgzZubyLG5FtZu7qlal4AaABAg,ItYOdWRo0JY,2019-08-01T15:45:49Z
2,recycle face,0,1,0,1,UgzSMwb88ntjkYHQYaN4AaABAg,ItYOdWRo0JY,2019-06-19T04:00:06Z
3,god dang twat,0,1,0,1,UgwS5xQLLnyIzNUS4bp4AaABAg,ItYOdWRo0JY,2019-06-16T00:49:53Z
4,not give away one 3 fans recognized you?,0,1,0,1,Ugz1EJ6K2F0CYUX5NrN4AaABAg,ItYOdWRo0JY,2019-03-31T00:50:58Z
5,trash trash stupid unsubscribing,0,1,0,1,UgyI5hLjBfIN99uVWDt4AaABAg,ItYOdWRo0JY,2019-03-16T03:50:36Z
6,crazy man kkk,0,1,0,1,UgwDghsKi_E8H3xYe-V4AaABAg,ItYOdWRo0JY,2019-02-22T17:48:07Z
7,physically hate,0,1,0,1,Ugxo2RWpZiVb6i0dGO54AaABAg,ItYOdWRo0JY,2019-02-04T15:31:57Z
8,first video see already fake. good job,0,1,0,1,UgwkHi8FfFqozu7c4KJ4AaABAg,ItYOdWRo0JY,2019-02-02T09:09:34Z
9,messed,0,1,0,1,Ugy-UnErAg23Btoph3x4AaABAg,ItYOdWRo0JY,2019-02-01T01:53:55Z


* replace emojis with their tags
* replace smileys ( ":)" etc.)
* consider if removing repeating consonants is a good idea
* remove punctuation
* find "haha" and replace with "laugh"
* stemming
* consider replacing insults into a tag, like "bad_word"
* tokenize

after that
* tf-idf for feature extraction? look into dictionaries: hownet, ntusd
* word2vec for embedding